In [1]:
import pandas as pd
import numpy as np
import time

In [1]:
class Team:
    def __init__(self, name):
        self.name = name
        self.points = 0
        self.goals_for = 0
        self.goals_against = 0
        self.wins = 0
        self.draws = 0
        self.losses = 0
        self.played = 0
        self.clean_sheets = 0
        
    def goals_scored_per_game(self):
        return round(self.goals_for/self.played,1)
    
    def goals_against_per_game(self):
        return round(self.goals_against/self.played,1)
    
    def points_per_game(self):
        return round(self.points/self.played,1)
    
    def add_result(self,result,isHome):
        [a_goals,b_goals] = [int(x) for x in result.split("-")]
        self.played += 1
        if isHome:
            self.goals_for += a_goals
            self.goals_against += b_goals
            if a_goals > b_goals:
                self.wins += 1
                self.points += 3
            elif a_goals == b_goals:
                self.draws += 1
                self.points += 1
            else:
                self.losses += 1
            if b_goals == 0:
                self.clean_sheets += 1
        else:
            self.goals_for += b_goals
            self.goals_against += a_goals
            if b_goals > a_goals:
                self.wins += 1
                self.points += 3
            elif b_goals == a_goals:
                self.draws += 1
                self.points += 1
            else:
                self.losses += 1
            if a_goals == 0:
                self.clean_sheets += 1

In [1]:
# make sure df sorted by date, date in pd.to_datetime format
# df_season['date'] = pd.to_datetime(df.date)
# df_season.sort_values(by='date',inplace=True,ascending=True)
# date is "date of next game"
def finalise_table(final_table):
    Pos = []
    Team = []
    GP = []
    W = []
    D = []
    L = []
    GS = []
    GC = []
    GD = []
    CS = []
    Pts = []
    for team in final_table.keys():
        team_obj = final_table.get(team)
        Pos.append(0)
        Team.append(team_obj.name)
        GP.append(team_obj.played)
        W.append(team_obj.wins)
        D.append(team_obj.draws)
        L.append(team_obj.losses)
        GS.append(team_obj.goals_for)
        GC.append(team_obj.goals_against)
        GD.append(team_obj.goals_for - team_obj.goals_against)
        CS.append(team_obj.clean_sheets)
        Pts.append(team_obj.points)
    table_as_dict = {'Pos':Pos,'Team':Team,'GP':GP,'W':W,'D':D,'L':L,'GS':GS,'GC':GC,'GD':GD,'CS':CS,'Pts':Pts}
    df = pd.DataFrame(table_as_dict)
    new_df = df.sort_values(by=['Pts','GD','GS'],axis=0,ascending=False)
    new_df.reset_index(drop=True,inplace=True)
    new_df['Pos'] = [x+1 for x in new_df.index]
    return new_df

def basic_league_table(date,df_season,teams):
    mask = (pd.to_datetime(df_season['date']) < pd.to_datetime(date))
    df_subset = df_season.loc[mask]
    
    table_dict = {}

    for team in sorted(teams):
        t1 = Team(team)
        df_team = df_subset.query("home == @team or away == @team")
        for i in range(len(df_team)):
            row = df_team.iloc[i]
            t1.add_result(row['score'],(row['home'] == team))
        table_dict[team] = t1
        
    final_table = finalise_table(table_dict)
    return final_table